In [52]:
import pandas as pd
import numpy as np
import itertools
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [53]:
data1 = pd.read_csv('Ames_Housing_Data.csv')
data1['CentralAir'].replace('Y',1,inplace=True)
data1['CentralAir'].replace('N',0,inplace=True)
print(data1.head())

   Id  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  CentralAir  \
0   1     8450            7            5       2003          2003           1   
1   2     9600            6            8       1976          1976           1   
2   3    11250            7            5       2001          2002           1   
3   4     9550            7            5       1915          1970           1   
4   5    14260            8            5       2000          2000           1   

   1stFlrSF  2ndFlrSF  GrLivArea  FullBath  HalfBath  BedroomAbvGr  \
0       856       854       1710         2         1             3   
1      1262         0       1262         2         0             3   
2       920       866       1786         2         1             3   
3       961       756       1717         1         0             3   
4      1145      1053       2198         2         1             4   

   KitchenAbvGr  TotRmsAbvGrd  Fireplaces  GarageCars  GarageArea  YrSold  \
0             1

In [39]:
#Question 1-1
x = data1.drop('SalePrice', axis=1)
y = data1['SalePrice']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=123)
print(x_train.shape)
print(x_test.shape)

(1095, 19)
(365, 19)
19


In [25]:
#Question 1-2
def forward_selection(x,y):
    initial_features = x.columns.tolist()
    best_features = []
    
lr = LinearRegression()
sfs = SequentialFeatureSelector(lr,n_features_to_select=np.log(x_train.shape[0]), direction = 'forward', cv = 5)
lr_sfs = LinearRegression()
stepwisemodel = lr_sfs.fit(x_test, y_test)
y_predict = stepwisemodel.predict(x_test)
stepwisemodel_MSE = mean_squared_error(y_test, y_predict)
stepwisemodel_R2 = r2_score(y_test, y_predict)
print("Stepwise Model intercept: ", stepwisemodel.intercept_)
print(stepwisemodel.coef_)
print('Stepwise Model MSE: ', stepwisemodel_MSE)
print('Stepwise Model R2: ', stepwisemodel_R2)

Stepwise Model intercept:  688251.1975948742
[-2.89310098e+00  6.81584629e-01  1.74391856e+04  9.20328226e+03
  4.70759516e+02 -3.32767528e+01 -9.31129224e+02  3.34233532e+00
 -3.23554552e+01  7.79684601e+01  7.17175234e+03  6.32144296e+03
 -1.41787052e+04 -2.71464923e+04  7.82816010e+03  3.96224907e+03
 -9.95864332e+03  7.02895371e+01 -8.24736625e+02]
Stepwise Model MSE:  1009059794.1088772
Stepwise Model R2:  0.8326735173740077


In [40]:
#Question 1-3
def bestSubset(x_test, y_test):
    best_model = None
    best_SSE = np.inf
    for k in range(1, x_test.shape[1]):
        for subset in itertools.combinations(x_test.columns, k):
           x_subset = x_train[list(subset)]
           model = LinearRegression()
           model.fit(x_subset, y_test)
           y_pred = model.predict(x_subset)
           SSE = np.sum((y_test-y_pred)**2)
           if SSE < best_SSE:
              best_SSE = SSE
              best_model = model
               
bestsubsetmodel = bestSubset(x_test, y_test)

print("Best subset model coefficients:", bestsubsetmodel.coef_)

ValueError: Found input variables with inconsistent numbers of samples: [1095, 365]

In [44]:
#Question 2-1
data2 = pd.read_csv('Titanic2.csv')
data2 = data2.drop(columns=['PassengerId','Name','Ticket','Cabin'])
data2 = data2.dropna()
print(data2.shape[0],'passengers left after deleting missing values in the variable age')
print(data2.head())

714 passengers left after deleting missing values in the variable age
   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S


In [45]:
#Question 2-2
print(data2.dtypes)
data2['Survived'] = data2['Survived'].astype('category')
data2['Pclass'] = data2['Pclass'].astype('category')
data2['Sex'].replace('male',0,inplace=True)
data2['Sex'].replace('female',1,inplace=True)
data2['Embarked'].replace('C',0,inplace=True)
data2['Embarked'].replace('Q',1,inplace=True)
data2['Embarked'].replace('S',2,inplace=True)
print(data2.head())

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object
  Survived Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0        0      3    0  22.0      1      0   7.2500         2
1        1      1    1  38.0      1      0  71.2833         0
2        1      3    1  26.0      0      0   7.9250         2
3        1      1    1  35.0      1      0  53.1000         2
4        0      3    0  35.0      0      0   8.0500         2


In [46]:
#Question 2-3
x = data2.drop('Survived',axis=1)
y = data2['Survived']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=123)
print(x_train.shape)
print(x_test.shape)

(571, 7)
(143, 7)


In [47]:
#Question 2-4
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [48]:
#Question 2-5
y_pred = model.predict(x_test)
print(y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix: ")
print(cm)
accuracy = accuracy_score(y_test,y_pred)
print("Overall accuracy: ", accuracy)
sensitivity = recall_score(y_test, y_pred)
print("Sensitivity score: ", sensitivity)
specificity = cm[0,0]/(cm[0,0]+cm[0,1])

[1 0 1 1 1 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 1 0
 0 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1]
Confusion matrix: 
[[69 18]
 [15 41]]
Overall accuracy:  0.7692307692307693
Sensitivity score:  0.7321428571428571


In [49]:
#Question 2-6
input_data = np.array([[3,0,23,0,0,8.25,1]])
prediction = model.predict(input_data)
print("Predicted: ", prediction)
# He is unlikely to survive. 

Predicted class:  [0]


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
